<a href="https://colab.research.google.com/github/ziwenwang28/QSVM_on_Synthetic_Datasets/blob/main/SVM_QSVM_on_Synthetic_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit_machine_learning
!pip install qiskit_aer
!pip install qiskit

In [17]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler


np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 5  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.48      0.70      0.57        20
           1       0.62      0.40      0.49        25

    accuracy                           0.53        45
   macro avg       0.55      0.55      0.53        45
weighted avg       0.56      0.53      0.52        45

Classical SVM training time with PCA: 0.0016090869903564453 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.36      0.25      0.29        20
           1       0.52      0.64      0.57        25

    accuracy                           0.47        45
   macro avg       0.44      0.45      0.43        45
weighted avg       0.45      0.47      0.45        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 52.478108406066895 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.40      0.40      0.40        20
           1       0.52      0.52      0.52        25

    accuracy                           0.47        45
   macro avg       0.46      0.46      0.46        45
weighted avg       0.47      0.47      0.47        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 24.673976182937622 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.45      0.43        20
           1       0.52      0.48      0.50        25

    accuracy                           0.47        45
   macro avg       0.47      0.46      0.46        45
weighted avg       0.47      0.47      0.47        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 42.653478384017944 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.43      0.60      0.50        20
           1       0.53      0.36      0.43        25

    accuracy                           0.47        45
   macro avg       0.48      0.48      0.46        45
weighted avg       0.48      0.47      0.46        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 107.62699055671692 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.36      0.40      0.38        20
           1       0.48      0.44      0.46        25

    accuracy                           0.42        45
   macro avg       0.42      0.42      0.42        45
weighted avg       0.43      0.42      0.42        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 128.72853136062622 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XYZ, Optimizer - SPSA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 5  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.48      0.70      0.57        20
           1       0.62      0.40      0.49        25

    accuracy                           0.53        45
   macro avg       0.55      0.55      0.53        45
weighted avg       0.56      0.53      0.52        45

Classical SVM training time with PCA: 0.0016355514526367188 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.35      0.35      0.35        20
           1       0.48      0.48      0.48        25

    accuracy                           0.42        45
   macro avg       0.41      0.41      0.41        45
weighted avg       0.42      0.42      0.42        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 55.339664697647095 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.32      0.30      0.31        20
           1       0.46      0.48      0.47        25

    accuracy                           0.40        45
   macro avg       0.39      0.39      0.39        45
weighted avg       0.40      0.40      0.40        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 32.8879189491272 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.45      0.43        20
           1       0.52      0.48      0.50        25

    accuracy                           0.47        45
   macro avg       0.47      0.46      0.46        45
weighted avg       0.47      0.47      0.47        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 44.29058337211609 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.55      0.49        20
           1       0.55      0.44      0.49        25

    accuracy                           0.49        45
   macro avg       0.49      0.49      0.49        45
weighted avg       0.50      0.49      0.49        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 128.3110659122467 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.40      0.50      0.44        20
           1       0.50      0.40      0.44        25

    accuracy                           0.44        45
   macro avg       0.45      0.45      0.44        45
weighted avg       0.46      0.44      0.44        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 95.9661693572998 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_YZ, Optimizer - COBYLA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.50      0.76      0.60        21
           1       0.62      0.33      0.43        24

    accuracy                           0.53        45
   macro avg       0.56      0.55      0.52        45
weighted avg       0.56      0.53      0.51        45

Classical SVM training time with PCA: 0.0016491413116455078 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.33      0.38        21
           1       0.52      0.62      0.57        24

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.47        45
weighted avg       0.48      0.49      0.48        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 53.73020792007446 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.56      0.71      0.63        21
           1       0.67      0.50      0.57        24

    accuracy                           0.60        45
   macro avg       0.61      0.61      0.60        45
weighted avg       0.61      0.60      0.60        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 31.521284341812134 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.33      0.37        21
           1       0.50      0.58      0.54        24

    accuracy                           0.47        45
   macro avg       0.46      0.46      0.45        45
weighted avg       0.46      0.47      0.46        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 33.41157031059265 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.57      0.50        21
           1       0.50      0.38      0.43        24

    accuracy                           0.47        45
   macro avg       0.47      0.47      0.46        45
weighted avg       0.47      0.47      0.46        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 194.65059661865234 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.29      0.35        21
           1       0.53      0.71      0.61        24

    accuracy                           0.51        45
   macro avg       0.50      0.50      0.48        45
weighted avg       0.50      0.51      0.49        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 94.72017788887024 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_YZ, Optimizer - L_BFGS_B


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.40      0.29      0.33        21
           1       0.50      0.62      0.56        24

    accuracy                           0.47        45
   macro avg       0.45      0.46      0.44        45
weighted avg       0.45      0.47      0.45        45

Classical SVM training time with PCA: 0.0015735626220703125 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.40      0.48      0.43        21
           1       0.45      0.38      0.41        24

    accuracy                           0.42        45
   macro avg       0.43      0.43      0.42        45
weighted avg       0.43      0.42      0.42        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 53.33926749229431 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.50      0.57      0.53        21
           1       0.57      0.50      0.53        24

    accuracy                           0.53        45
   macro avg       0.54      0.54      0.53        45
weighted avg       0.54      0.53      0.53        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 30.402641773223877 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.33      0.38      0.36        21
           1       0.38      0.33      0.36        24

    accuracy                           0.36        45
   macro avg       0.36      0.36      0.36        45
weighted avg       0.36      0.36      0.36        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 33.19159150123596 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.53      0.38      0.44        21
           1       0.57      0.71      0.63        24

    accuracy                           0.56        45
   macro avg       0.55      0.54      0.54        45
weighted avg       0.55      0.56      0.54        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 74.4995129108429 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.33      0.39        21
           1       0.53      0.67      0.59        24

    accuracy                           0.51        45
   macro avg       0.50      0.50      0.49        45
weighted avg       0.50      0.51      0.50        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 58.61647701263428 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XY, Optimizer - L_BFGS_B


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.40      0.29      0.33        21
           1       0.50      0.62      0.56        24

    accuracy                           0.47        45
   macro avg       0.45      0.46      0.44        45
weighted avg       0.45      0.47      0.45        45

Classical SVM training time with PCA: 0.0015692710876464844 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.43      0.42        21
           1       0.48      0.46      0.47        24

    accuracy                           0.44        45
   macro avg       0.44      0.44      0.44        45
weighted avg       0.45      0.44      0.44        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 54.41015124320984 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.59      0.62      0.60        21
           1       0.65      0.62      0.64        24

    accuracy                           0.62        45
   macro avg       0.62      0.62      0.62        45
weighted avg       0.62      0.62      0.62        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 29.24331545829773 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.55      0.57      0.56        21
           1       0.61      0.58      0.60        24

    accuracy                           0.58        45
   macro avg       0.58      0.58      0.58        45
weighted avg       0.58      0.58      0.58        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 37.28031253814697 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.38      0.41        21
           1       0.52      0.58      0.55        24

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.48        45
weighted avg       0.48      0.49      0.48        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 119.43451356887817 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.50      0.33      0.40        21
           1       0.55      0.71      0.62        24

    accuracy                           0.53        45
   macro avg       0.52      0.52      0.51        45
weighted avg       0.53      0.53      0.52        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 79.02760481834412 seconds
Best Configuration with PCA: Feature Map - ZZFeatureMap, Optimizer - COBYLA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.50      0.20      0.29        25
           1       0.43      0.75      0.55        20

    accuracy                           0.44        45
   macro avg       0.46      0.47      0.42        45
weighted avg       0.47      0.44      0.40        45

Classical SVM training time with PCA: 0.0017189979553222656 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.44      0.45        25
           1       0.33      0.35      0.34        20

    accuracy                           0.40        45
   macro avg       0.40      0.40      0.40        45
weighted avg       0.40      0.40      0.40        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 52.05704402923584 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.32      0.37        25
           1       0.37      0.50      0.43        20

    accuracy                           0.40        45
   macro avg       0.41      0.41      0.40        45
weighted avg       0.41      0.40      0.40        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 27.919809818267822 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.28      0.33        25
           1       0.36      0.50      0.42        20

    accuracy                           0.38        45
   macro avg       0.38      0.39      0.38        45
weighted avg       0.39      0.38      0.37        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 34.551055908203125 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.32      0.38        25
           1       0.39      0.55      0.46        20

    accuracy                           0.42        45
   macro avg       0.43      0.44      0.42        45
weighted avg       0.44      0.42      0.42        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 155.34376740455627 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.56      0.36      0.44        25
           1       0.45      0.65      0.53        20

    accuracy                           0.49        45
   macro avg       0.51      0.51      0.48        45
weighted avg       0.51      0.49      0.48        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 171.5480556488037 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XYZ, Optimizer - COBYLA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.5  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.42      0.20      0.27        25
           1       0.39      0.65      0.49        20

    accuracy                           0.40        45
   macro avg       0.41      0.43      0.38        45
weighted avg       0.41      0.40      0.37        45

Classical SVM training time with PCA: 0.0016932487487792969 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.40      0.40      0.40        25
           1       0.25      0.25      0.25        20

    accuracy                           0.33        45
   macro avg       0.33      0.33      0.33        45
weighted avg       0.33      0.33      0.33        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 54.31480431556702 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.52      0.56      0.54        25
           1       0.39      0.35      0.37        20

    accuracy                           0.47        45
   macro avg       0.45      0.46      0.45        45
weighted avg       0.46      0.47      0.46        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 26.093300342559814 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.56      0.55        25
           1       0.42      0.40      0.41        20

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.48        45
weighted avg       0.49      0.49      0.49        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 44.49374175071716 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.59      0.52      0.55        25
           1       0.48      0.55      0.51        20

    accuracy                           0.53        45
   macro avg       0.53      0.54      0.53        45
weighted avg       0.54      0.53      0.53        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 136.70653533935547 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.56      0.55        25
           1       0.42      0.40      0.41        20

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.48        45
weighted avg       0.49      0.49      0.49        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 96.77875900268555 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XYZ, Optimizer - SPSA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.42      0.20      0.27        25
           1       0.39      0.65      0.49        20

    accuracy                           0.40        45
   macro avg       0.41      0.43      0.38        45
weighted avg       0.41      0.40      0.37        45

Classical SVM training time with PCA: 0.0016565322875976562 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.57      0.52      0.54        25
           1       0.45      0.50      0.48        20

    accuracy                           0.51        45
   macro avg       0.51      0.51      0.51        45
weighted avg       0.52      0.51      0.51        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 54.06768345832825 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.56      0.55        25
           1       0.42      0.40      0.41        20

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.48        45
weighted avg       0.49      0.49      0.49        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 31.967506408691406 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.52      0.49        25
           1       0.29      0.25      0.27        20

    accuracy                           0.40        45
   macro avg       0.38      0.39      0.38        45
weighted avg       0.39      0.40      0.39        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 38.42970299720764 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.52      0.48      0.50        25
           1       0.41      0.45      0.43        20

    accuracy                           0.47        45
   macro avg       0.47      0.46      0.46        45
weighted avg       0.47      0.47      0.47        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 120.45731353759766 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.56      0.55        25
           1       0.42      0.40      0.41        20

    accuracy                           0.49        45
   macro avg       0.48      0.48      0.48        45
weighted avg       0.49      0.49      0.49        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 158.56732630729675 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XYZ, Optimizer - COBYLA


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 150
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.42      0.20      0.27        25
           1       0.39      0.65      0.49        20

    accuracy                           0.40        45
   macro avg       0.41      0.43      0.38        45
weighted avg       0.41      0.40      0.37        45

Classical SVM training time with PCA: 0.0015926361083984375 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.58      0.56      0.57        25
           1       0.48      0.50      0.49        20

    accuracy                           0.53        45
   macro avg       0.53      0.53      0.53        45
weighted avg       0.54      0.53      0.53        45

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 54.109352350234985 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.56      0.56      0.56        25
           1       0.45      0.45      0.45        20

    accuracy                           0.51        45
   macro avg       0.51      0.51      0.51        45
weighted avg       0.51      0.51      0.51        45

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 30.16848349571228 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.59      0.52      0.55        25
           1       0.48      0.55      0.51        20

    accuracy                           0.53        45
   macro avg       0.53      0.54      0.53        45
weighted avg       0.54      0.53      0.53        45

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 39.35039210319519 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.52      0.52      0.52        25
           1       0.40      0.40      0.40        20

    accuracy                           0.47        45
   macro avg       0.46      0.46      0.46        45
weighted avg       0.47      0.47      0.47        45

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 73.63667798042297 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.63      0.48      0.55        25
           1       0.50      0.65      0.57        20

    accuracy                           0.56        45
   macro avg       0.57      0.56      0.56        45
weighted avg       0.57      0.56      0.55        45

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 40.56773805618286 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_ZZ, Optimizer - SLSQP


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 5  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.53      0.72      0.61        47
           1       0.50      0.30      0.38        43

    accuracy                           0.52        90
   macro avg       0.52      0.51      0.49        90
weighted avg       0.52      0.52      0.50        90

Classical SVM training time with PCA: 0.0041964054107666016 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.53      0.64      0.58        47
           1       0.48      0.37      0.42        43

    accuracy                           0.51        90
   macro avg       0.51      0.51      0.50        90
weighted avg       0.51      0.51      0.50        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 108.43597722053528 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.56      0.74      0.64        47
           1       0.56      0.35      0.43        43

    accuracy                           0.56        90
   macro avg       0.56      0.55      0.53        90
weighted avg       0.56      0.56      0.54        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 56.580108642578125 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.63      0.47      0.54        47
           1       0.55      0.70      0.61        43

    accuracy                           0.58        90
   macro avg       0.59      0.58      0.57        90
weighted avg       0.59      0.58      0.57        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 73.91220235824585 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.62      0.57        47
           1       0.50      0.42      0.46        43

    accuracy                           0.52        90
   macro avg       0.52      0.52      0.51        90
weighted avg       0.52      0.52      0.52        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 417.0975458621979 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.55      0.72      0.62        47
           1       0.54      0.35      0.42        43

    accuracy                           0.54        90
   macro avg       0.54      0.54      0.52        90
weighted avg       0.54      0.54      0.53        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 190.40241289138794 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_Y, Optimizer - SPSA


In [9]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 5  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.53      0.72      0.61        47
           1       0.50      0.30      0.38        43

    accuracy                           0.52        90
   macro avg       0.52      0.51      0.49        90
weighted avg       0.52      0.52      0.50        90

Classical SVM training time with PCA: 0.0049741268157958984 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.56      0.68      0.62        47
           1       0.55      0.42      0.47        43

    accuracy                           0.56        90
   macro avg       0.55      0.55      0.54        90
weighted avg       0.55      0.56      0.55        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 108.81609058380127 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.53      0.62      0.57        47
           1       0.49      0.40      0.44        43

    accuracy                           0.51        90
   macro avg       0.51      0.51      0.50        90
weighted avg       0.51      0.51      0.51        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 71.03211307525635 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.61      0.47      0.53        47
           1       0.54      0.67      0.60        43

    accuracy                           0.57        90
   macro avg       0.57      0.57      0.56        90
weighted avg       0.58      0.57      0.56        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 71.74887442588806 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.45      0.45        47
           1       0.41      0.42      0.41        43

    accuracy                           0.43        90
   macro avg       0.43      0.43      0.43        90
weighted avg       0.43      0.43      0.43        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 179.85354590415955 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.51      0.49      0.50        47
           1       0.47      0.49      0.48        43

    accuracy                           0.49        90
   macro avg       0.49      0.49      0.49        90
weighted avg       0.49      0.49      0.49        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 95.07203125953674 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_Y, Optimizer - SPSA


In [10]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.37      0.55      0.44        40
           1       0.40      0.24      0.30        50

    accuracy                           0.38        90
   macro avg       0.38      0.40      0.37        90
weighted avg       0.39      0.38      0.36        90

Classical SVM training time with PCA: 0.0029418468475341797 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.42      0.40      0.41        40
           1       0.54      0.56      0.55        50

    accuracy                           0.49        90
   macro avg       0.48      0.48      0.48        90
weighted avg       0.49      0.49      0.49        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 107.5905990600586 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        40
           1       0.66      0.66      0.66        50

    accuracy                           0.62        90
   macro avg       0.62      0.62      0.62        90
weighted avg       0.62      0.62      0.62        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 67.71854043006897 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.50      0.45        40
           1       0.51      0.42      0.46        50

    accuracy                           0.46        90
   macro avg       0.46      0.46      0.46        90
weighted avg       0.47      0.46      0.46        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 73.89952325820923 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.47      0.44        40
           1       0.52      0.46      0.49        50

    accuracy                           0.47        90
   macro avg       0.47      0.47      0.47        90
weighted avg       0.47      0.47      0.47        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 425.64698219299316 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.43      0.50      0.47        40
           1       0.55      0.48      0.51        50

    accuracy                           0.49        90
   macro avg       0.49      0.49      0.49        90
weighted avg       0.50      0.49      0.49        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 215.80100798606873 seconds
Best Configuration with PCA: Feature Map - ZZFeatureMap, Optimizer - COBYLA


In [11]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.33      0.28      0.30        40
           1       0.49      0.56      0.52        50

    accuracy                           0.43        90
   macro avg       0.41      0.42      0.41        90
weighted avg       0.42      0.43      0.42        90

Classical SVM training time with PCA: 0.006456136703491211 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.43      0.50      0.46        40
           1       0.53      0.46      0.49        50

    accuracy                           0.48        90
   macro avg       0.48      0.48      0.48        90
weighted avg       0.49      0.48      0.48        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 105.74257636070251 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.38      0.45      0.41        40
           1       0.48      0.40      0.43        50

    accuracy                           0.42        90
   macro avg       0.43      0.43      0.42        90
weighted avg       0.43      0.42      0.42        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 63.6383593082428 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.42      0.33      0.37        40
           1       0.54      0.64      0.59        50

    accuracy                           0.50        90
   macro avg       0.48      0.48      0.48        90
weighted avg       0.49      0.50      0.49        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 79.12740898132324 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.41      0.38      0.39        40
           1       0.53      0.56      0.54        50

    accuracy                           0.48        90
   macro avg       0.47      0.47      0.47        90
weighted avg       0.47      0.48      0.48        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 159.25926446914673 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.37      0.35      0.36        40
           1       0.50      0.52      0.51        50

    accuracy                           0.44        90
   macro avg       0.43      0.43      0.43        90
weighted avg       0.44      0.44      0.44        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 168.69717979431152 seconds
Best Configuration with PCA: Feature Map - ZZFeatureMap, Optimizer - SLSQP


In [12]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 7  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.33      0.28      0.30        40
           1       0.49      0.56      0.52        50

    accuracy                           0.43        90
   macro avg       0.41      0.42      0.41        90
weighted avg       0.42      0.43      0.42        90

Classical SVM training time with PCA: 0.0028040409088134766 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.43      0.50      0.47        40
           1       0.55      0.48      0.51        50

    accuracy                           0.49        90
   macro avg       0.49      0.49      0.49        90
weighted avg       0.50      0.49      0.49        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 105.42980432510376 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.39      0.50      0.44        40
           1       0.49      0.38      0.43        50

    accuracy                           0.43        90
   macro avg       0.44      0.44      0.43        90
weighted avg       0.44      0.43      0.43        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 60.4544575214386 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.38      0.42        40
           1       0.57      0.66      0.61        50

    accuracy                           0.53        90
   macro avg       0.52      0.52      0.51        90
weighted avg       0.52      0.53      0.52        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 76.43242001533508 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.38      0.40      0.39        40
           1       0.50      0.48      0.49        50

    accuracy                           0.44        90
   macro avg       0.44      0.44      0.44        90
weighted avg       0.45      0.44      0.45        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 318.5445306301117 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.35      0.42      0.38        40
           1       0.44      0.36      0.40        50

    accuracy                           0.39        90
   macro avg       0.39      0.39      0.39        90
weighted avg       0.40      0.39      0.39        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 151.9528567790985 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_YZ, Optimizer - SPSA


In [13]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.53      0.20      0.29        40
           1       0.57      0.86      0.69        50

    accuracy                           0.57        90
   macro avg       0.55      0.53      0.49        90
weighted avg       0.56      0.57      0.51        90

Classical SVM training time with PCA: 0.002827882766723633 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.47      0.48        40
           1       0.58      0.58      0.58        50

    accuracy                           0.53        90
   macro avg       0.53      0.53      0.53        90
weighted avg       0.53      0.53      0.53        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 104.59419655799866 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.50      0.38      0.43        40
           1       0.58      0.70      0.64        50

    accuracy                           0.56        90
   macro avg       0.54      0.54      0.53        90
weighted avg       0.55      0.56      0.54        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 67.50405812263489 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.15      0.23        40
           1       0.56      0.86      0.68        50

    accuracy                           0.54        90
   macro avg       0.51      0.51      0.45        90
weighted avg       0.52      0.54      0.48        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 71.87928318977356 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.54      0.33      0.41        40
           1       0.59      0.78      0.67        50

    accuracy                           0.58        90
   macro avg       0.57      0.55      0.54        90
weighted avg       0.57      0.58      0.55        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 265.486453294754 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.58      0.38      0.45        40
           1       0.61      0.78      0.68        50

    accuracy                           0.60        90
   macro avg       0.59      0.58      0.57        90
weighted avg       0.59      0.60      0.58        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 40.84387540817261 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_XYZYY, Optimizer - SLSQP


In [14]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 0.5  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.31      0.12      0.18        40
           1       0.53      0.78      0.63        50

    accuracy                           0.49        90
   macro avg       0.42      0.45      0.40        90
weighted avg       0.43      0.49      0.43        90

Classical SVM training time with PCA: 0.0029010772705078125 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.44      0.47      0.46        40
           1       0.55      0.52      0.54        50

    accuracy                           0.50        90
   macro avg       0.50      0.50      0.50        90
weighted avg       0.50      0.50      0.50        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 104.54405069351196 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.52      0.35      0.42        40
           1       0.59      0.74      0.65        50

    accuracy                           0.57        90
   macro avg       0.55      0.54      0.54        90
weighted avg       0.56      0.57      0.55        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 57.84174346923828 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.38      0.23      0.28        40
           1       0.53      0.70      0.60        50

    accuracy                           0.49        90
   macro avg       0.45      0.46      0.44        90
weighted avg       0.46      0.49      0.46        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 72.86744737625122 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.46      0.30      0.36        40
           1       0.56      0.72      0.63        50

    accuracy                           0.53        90
   macro avg       0.51      0.51      0.50        90
weighted avg       0.52      0.53      0.51        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 423.3273310661316 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.35      0.17      0.23        40
           1       0.53      0.74      0.62        50

    accuracy                           0.49        90
   macro avg       0.44      0.46      0.42        90
weighted avg       0.45      0.49      0.45        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 278.1522305011749 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_YZ, Optimizer - L_BFGS_B


In [15]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 2.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.31      0.12      0.18        40
           1       0.53      0.78      0.63        50

    accuracy                           0.49        90
   macro avg       0.42      0.45      0.40        90
weighted avg       0.43      0.49      0.43        90

Classical SVM training time with PCA: 0.0029981136322021484 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.55      0.51        40
           1       0.58      0.50      0.54        50

    accuracy                           0.52        90
   macro avg       0.52      0.53      0.52        90
weighted avg       0.53      0.52      0.52        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 105.23830771446228 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.52      0.33      0.40        40
           1       0.58      0.76      0.66        50

    accuracy                           0.57        90
   macro avg       0.55      0.54      0.53        90
weighted avg       0.56      0.57      0.54        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 50.61063480377197 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.50      0.25      0.33        40
           1       0.57      0.80      0.67        50

    accuracy                           0.56        90
   macro avg       0.54      0.53      0.50        90
weighted avg       0.54      0.56      0.52        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 69.8808023929596 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.22      0.05      0.08        40
           1       0.53      0.86      0.66        50

    accuracy                           0.50        90
   macro avg       0.38      0.46      0.37        90
weighted avg       0.39      0.50      0.40        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 245.54665207862854 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.43      0.33      0.37        40
           1       0.55      0.66      0.60        50

    accuracy                           0.51        90
   macro avg       0.49      0.49      0.49        90
weighted avg       0.50      0.51      0.50        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 188.2168848514557 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_ZZ, Optimizer - SLSQP


In [16]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from qiskit import Aer
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP, ADAM
from time import time

np.random.seed(0)

# Define the number of samples and features
num_samples = 300
num_features = 10  # You can increase this number to create a higher-dimensional dataset

# Generate random data with strong correlations in the first few features
mean = np.zeros(num_features)
cov = np.eye(num_features)
for i in range(5):
    cov[i, i] = 10.0  # Increase the variance for the first 5 features to make them important

data = np.random.multivariate_normal(mean, cov, num_samples)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, np.random.choice([0, 1], size=num_samples), test_size=0.3, random_state=42)

# Data Preprocessing
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classical SVM on reduced dataset
classical_svm_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
start = time()
classical_svm_pca.fit(X_train_pca, y_train)
end = time()
classical_svm_predictions_pca = classical_svm_pca.predict(X_test_pca)
print("Performance Metrics for Classical SVM with PCA:")
print(classification_report(y_test, classical_svm_predictions_pca))
print(f"Classical SVM training time with PCA: {end - start} seconds")

# Define a list of feature maps and their parameters to tune for QSVM
feature_maps = [
    {'name': 'ZZFeatureMap', 'feature_map': ZZFeatureMap(feature_dimension=2, reps=2, entanglement='full')},
    {'name': 'PauliFeatureMap_XY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_ZZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Z', 'ZZ'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_YZ', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y', 'Z'], entanglement='full')},
    {'name': 'PauliFeatureMap_Y', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['Y'], entanglement='full')},
    {'name': 'PauliFeatureMap_XYZYY', 'feature_map': PauliFeatureMap(feature_dimension=2, reps=2, paulis=['X', 'Y', 'Z', 'YY'], entanglement='full')},
    # Add more feature maps and their parameters to explore
]

# Define a list of optimizer parameters to tune for QSVM
optimizers = [
    {'name': 'SPSA', 'optimizer': SPSA(maxiter=50)},
    {'name': 'COBYLA', 'optimizer': COBYLA(maxiter=100)},
    {'name': 'L_BFGS_B', 'optimizer': L_BFGS_B(maxfun=100, maxiter=100)},
    {'name': 'SLSQP', 'optimizer': SLSQP(maxiter=100)},
    # Add more optimizers and their parameters to explore
]


best_score = 0
best_params = {}

for feature_map_params in feature_maps:
    feature_map_name = feature_map_params['name']
    feature_map = feature_map_params['feature_map']

    for optimizer_params in optimizers:
        optimizer_name = optimizer_params['name']
        optimizer = optimizer_params['optimizer']

        # Create a sampler for the AerSimulator
        sampler = Sampler()

        # Create an instance of the AerSimulator
        simulator = AerSimulator()

        # Train QSVM with the current feature map and optimizer using the sampler
        vqc = VQC(feature_map=feature_map, ansatz=None, optimizer=optimizer, quantum_instance=simulator)

        start = time()
        vqc.fit(X_train_pca, y_train)
        end = time()

        result_quantum = vqc.predict(X_test_pca)
        score = accuracy_score(y_test, result_quantum)

        # Print performance metrics and runtime for the current configuration
        print(f"Performance Metrics for Quantum SVM ({feature_map_name}, {optimizer_name}) with PCA:")
        print(classification_report(y_test, result_quantum))
        print(f"Quantum SVM training time ({feature_map_name}, {optimizer_name}) with PCA: {end - start} seconds")

        # Check if the current configuration is the best
        if score > best_score:
            best_score = score
            best_params = {'feature_map': feature_map_name, 'optimizer': optimizer_name}

# Print the best configuration found
print(f"Best Configuration with PCA: Feature Map - {best_params['feature_map']}, Optimizer - {best_params['optimizer']}")

Performance Metrics for Classical SVM with PCA:
              precision    recall  f1-score   support

           0       0.31      0.12      0.18        40
           1       0.53      0.78      0.63        50

    accuracy                           0.49        90
   macro avg       0.42      0.45      0.40        90
weighted avg       0.43      0.49      0.43        90

Classical SVM training time with PCA: 0.0030121803283691406 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, SPSA) with PCA:
              precision    recall  f1-score   support

           0       0.47      0.35      0.40        40
           1       0.57      0.68      0.62        50

    accuracy                           0.53        90
   macro avg       0.52      0.52      0.51        90
weighted avg       0.52      0.53      0.52        90

Quantum SVM training time (ZZFeatureMap, SPSA) with PCA: 104.94379925727844 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (ZZFeatureMap, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.50      0.38      0.43        40
           1       0.58      0.70      0.64        50

    accuracy                           0.56        90
   macro avg       0.54      0.54      0.53        90
weighted avg       0.55      0.56      0.54        90

Quantum SVM training time (ZZFeatureMap, COBYLA) with PCA: 51.52410817146301 seconds


...

Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, COBYLA) with PCA:
              precision    recall  f1-score   support

           0       0.37      0.17      0.24        40
           1       0.54      0.76      0.63        50

    accuracy                           0.50        90
   macro avg       0.45      0.47      0.43        90
weighted avg       0.46      0.50      0.45        90

Quantum SVM training time (PauliFeatureMap_XYZYY, COBYLA) with PCA: 87.49781632423401 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA:
              precision    recall  f1-score   support

           0       0.38      0.23      0.28        40
           1       0.53      0.70      0.60        50

    accuracy                           0.49        90
   macro avg       0.45      0.46      0.44        90
weighted avg       0.46      0.49      0.46        90

Quantum SVM training time (PauliFeatureMap_XYZYY, L_BFGS_B) with PCA: 440.58370780944824 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Performance Metrics for Quantum SVM (PauliFeatureMap_XYZYY, SLSQP) with PCA:
              precision    recall  f1-score   support

           0       0.36      0.20      0.26        40
           1       0.53      0.72      0.61        50

    accuracy                           0.49        90
   macro avg       0.45      0.46      0.43        90
weighted avg       0.46      0.49      0.45        90

Quantum SVM training time (PauliFeatureMap_XYZYY, SLSQP) with PCA: 294.2910511493683 seconds
Best Configuration with PCA: Feature Map - PauliFeatureMap_YZ, Optimizer - SLSQP
